# CBSA to SQLite Table

## Step By Step

In [1]:
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm

In [2]:
pd.set_option("display.max_columns", 500)
pd.set_option('display.max_rows', 1000)

**Read in the cbsa zip file (ZIP_CBSA_122020.csv) and assign zip codes to the Nashville CBSA**

In [3]:
zips_raw = pd.read_excel("../data/ZIP_CBSA_122020.xlsx", converters={'zip': lambda x: str(x)}, engine='openpyxl')

In [4]:
display(zips_raw.shape)
display(zips_raw)

(47424, 6)

,ZIP,CBSA,RES_RATIO,BUS_RATIO,OTH_RATIO,TOT_RATIO
0,501,35620,0.0,1.0,0.0,1.0
1,601,38660,1.0,1.0,1.0,1.0
2,602,10380,1.0,1.0,1.0,1.0
3,603,10380,1.0,1.0,1.0,1.0
4,604,10380,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...
47419,99925,99999,0.0,0.0,1.0,1.0
47420,99926,99999,0.0,0.0,1.0,1.0
47421,99927,99999,0.0,0.0,1.0,1.0
47422,99928,28540,0.0,0.0,1.0,1.0


**Rename Columns**

In [5]:
zips = zips_raw.reset_index().rename({
    'ZIP': 'zip',
    'CBSA': 'cbsa',
    'TOT_RATIO': 'tot_ratio'
}, axis=1)

**Dropping unneeded columns**

In [6]:
zips = zips.drop(['index', 'RES_RATIO', 'BUS_RATIO', 'OTH_RATIO'], axis=1)

In [7]:
display(zips.shape)
display(zips)

(47424, 3)

,zip,cbsa,tot_ratio
0,501,35620,1.0
1,601,38660,1.0
2,602,10380,1.0
3,603,10380,1.0
4,604,10380,1.0
...,...,...,...
47419,99925,99999,1.0
47420,99926,99999,1.0
47421,99927,99999,1.0
47422,99928,28540,1.0


**Return all zip codes and cbsa IDs WHERE the zip code tot_ratio is highest for the selected CBSA AND the cbsa = 34980 (Nashville). Temporary Loading to SQL to accomplish this.**

In [8]:
with sqlite3.connect('../data/hcbb_group_reviews.sqlite') as db:
   
    # Temporarily load to sqlite db
    zips.to_sql('temp_zips', db, if_exists = 'replace', index = False)
    
    # Use ranking and filtering
    query = ''' 
    WITH ratio AS (
        SELECT 
            zip, 
            cbsa, 
            RANK() OVER(PARTITION BY zip ORDER BY tot_ratio DESC) AS rnk 
        FROM temp_zips
    )
    SELECT 
        zip, 
        cbsa
    FROM ratio
    WHERE rnk = 1 AND cbsa = '34980';
    '''

    # Get the dataframe
    cbsa_df = pd.read_sql(query, db)
    
    # Delete the temporary table
    cursor = db.cursor()
    cursor.execute("DROP TABLE temp_zips")
    print("Temporary table dropped...")
    
    # Finally, load to final sqlite db
    cbsa_df.to_sql('cbsa', db, if_exists = 'replace', index = False)
    
    print("cbsa table created.")

Temporary table dropped...
cbsa table created.


## Testing Final DB Load

In [9]:
# Get list of currently existing tables
with sqlite3.connect('../data/hcbb_group_reviews.sqlite') as db :
    query = """
    SELECT name
    FROM sqlite_master 
    WHERE type ='table' 
    AND name NOT LIKE 'sqlite_%';
    """ 
    
    db_table_list = pd.read_sql(query, db)

display(db_table_list)

,name
0,npidata
1,cbsa


**This should return 135 rows each representing a unique zip code within the Nashville cbsa**

In [10]:
with sqlite3.connect('../data/hcbb_group_reviews.sqlite') as db :
    query = """
    SELECT COUNT(DISTINCT(zip))
    FROM cbsa;
    """ 
    
    test_df = pd.read_sql(query, db)

display(test_df)

,COUNT(DISTINCT(zip))
0,135
